# NBA Championship Predictor - Data Acquisition

This notebook demonstrates the data acquisition process for the NBA Championship Predictor project.

## Overview
- **Data Source**: Kaggle NBA Basketball Database by wyattowalsh
- **Size**: 697MB compressed, 60,192+ regular season games
- **Time Range**: 1946-2023 (77 seasons)
- **Format**: SQLite database with 16 tables

## 1. Setup and Installation

First, ensure you have the necessary packages installed:

In [ ]:
# Install required packages
!pip install kagglehub pandas numpy sqlite3

## 2. Import Required Libraries

In [ ]:
import os
import kagglehub
import shutil
import sqlite3
import pandas as pd
import numpy as np
import json
from pathlib import Path

print("Libraries imported successfully")

## 3. Configure Kaggle API

**Note**: You need Kaggle API credentials to download the dataset.

### Setup Instructions:
1. Go to https://www.kaggle.com/
2. Navigate to Account Settings → API → Create New Token
3. Download `kaggle.json`
4. Place it in:
   - **Windows**: `C:\Users\YourName\.kaggle\kaggle.json`
   - **Linux/Mac**: `~/.kaggle/kaggle.json`

In [ ]:
# Check if Kaggle credentials are configured
kaggle_dir = Path.home() / '.kaggle'
kaggle_json = kaggle_dir / 'kaggle.json'

if kaggle_json.exists():
    print("✓ Kaggle API credentials found")
else:
    print("✗ Kaggle API credentials not found")
    print(f"  Please place kaggle.json at: {kaggle_json}")

## 4. Download NBA Dataset

Download the NBA Basketball Database from Kaggle using kagglehub.

In [ ]:
print("Downloading NBA Basketball Dataset from Kaggle...")
print("This may take a few minutes (697MB compressed)")
print()

# Download the dataset
dataset_path = kagglehub.dataset_download("wyattowalsh/basketball")
print(f"Dataset downloaded to: {dataset_path}")

## 5. Copy Database to Project Directory

In [ ]:
# Create data directory in project
data_dir = "data"
os.makedirs(data_dir, exist_ok=True)

# Copy database file
db_source = os.path.join(dataset_path, "nba.sqlite")
db_dest = os.path.join(data_dir, "nba.sqlite")

if os.path.exists(db_source):
    print(f"Copying database to {db_dest}")
    shutil.copy2(db_source, db_dest)
    
    # Get file size
    size_bytes = os.path.getsize(db_dest)
    size_gb = size_bytes / (1024**3)
    print(f"✓ Database setup complete! ({size_gb:.2f} GB)")
else:
    print("✗ Database file not found in downloaded dataset")

## 6. Create Configuration File

In [ ]:
# Save configuration
config = {
    "dataset_path": dataset_path.replace('\\', '/'),
    "db_path": "data/nba.sqlite"
}

config_path = os.path.join(data_dir, "config.json")
with open(config_path, 'w') as f:
    json.dump(config, f, indent=2)
    
print(f"✓ Configuration saved to {config_path}")

## 7. Explore the Database

Let's examine the database structure and available data.

In [ ]:
# Connect to database
conn = sqlite3.connect(db_dest)

# Get list of all tables
tables_query = "SELECT name FROM sqlite_master WHERE type='table' ORDER BY name;"
tables = pd.read_sql_query(tables_query, conn)

print("Available Tables in Database:")
print("=" * 50)
for idx, table in enumerate(tables['name'], 1):
    print(f"{idx:2d}. {table}")

## 8. Examine Key Tables

### 8.1 Game Table (Primary Data Source)

In [ ]:
# Get game table info
game_info = pd.read_sql_query("PRAGMA table_info(game);", conn)
print("Game Table Columns:")
print(game_info[['name', 'type']].to_string(index=False))

In [ ]:
# Count total games
total_games = pd.read_sql_query("SELECT COUNT(*) as count FROM game;", conn)
print(f"Total Games in Database: {total_games['count'][0]:,}")

# Count regular season games
regular_season = pd.read_sql_query(
    "SELECT COUNT(*) as count FROM game WHERE season_type = 'Regular Season';", 
    conn
)
print(f"Regular Season Games: {regular_season['count'][0]:,}")

### 8.2 Team Table

In [ ]:
# Get all teams
teams = pd.read_sql_query(
    "SELECT id, full_name, abbreviation, city FROM team ORDER BY full_name;", 
    conn
)

print(f"Total Teams: {len(teams)}")
print("\nNBA Teams:")
print(teams.to_string(index=False))

### 8.3 Season Data Range

In [ ]:
# Get season range
seasons = pd.read_sql_query(
    """SELECT 
        MIN(season_id) as first_season,
        MAX(season_id) as last_season,
        COUNT(DISTINCT season_id) as total_seasons
       FROM game
       WHERE season_type = 'Regular Season';""",
    conn
)

print("Season Data Range:")
print(f"First Season: {seasons['first_season'][0]}")
print(f"Last Season: {seasons['last_season'][0]}")
print(f"Total Seasons: {seasons['total_seasons'][0]}")

### 8.4 Sample Game Data

In [ ]:
# Get sample games from recent season
sample_games = pd.read_sql_query(
    """SELECT 
        game_id,
        season_id,
        game_date,
        team_id_home,
        team_id_away,
        pts_home,
        pts_away,
        fg_pct_home,
        fg_pct_away,
        reb_home,
        reb_away
       FROM game 
       WHERE season_type = 'Regular Season' 
         AND season_id = 22022
       LIMIT 5;""",
    conn
)

print("Sample Games (2021-22 Season):")
print(sample_games)

### 8.5 Other Stats Table (Advanced Metrics)

In [ ]:
# Check other_stats table
other_stats_info = pd.read_sql_query("PRAGMA table_info(other_stats);", conn)
print("Other Stats Table Columns (Advanced Metrics):")
print(other_stats_info[['name', 'type']].to_string(index=False))

In [ ]:
# Sample advanced stats
advanced_stats = pd.read_sql_query(
    """SELECT 
        game_id,
        pts_paint_home,
        pts_paint_away,
        pts_2nd_chance_home,
        pts_2nd_chance_away,
        pts_fb_home,
        pts_fb_away,
        pts_off_to_home,
        pts_off_to_away
       FROM other_stats 
       LIMIT 5;""",
    conn
)

print("Sample Advanced Stats:")
print(advanced_stats)

## 9. Data Quality Check

Verify data quality for our analysis period (2003-2022).

In [ ]:
# Check data completeness for analysis period
analysis_data = pd.read_sql_query(
    """SELECT 
        season_id,
        COUNT(*) as game_count,
        COUNT(DISTINCT team_id_home) + COUNT(DISTINCT team_id_away) as unique_teams
       FROM game
       WHERE season_type = 'Regular Season'
         AND season_id >= 22003
         AND season_id <= 22022
       GROUP BY season_id
       ORDER BY season_id;""",
    conn
)

print("Games per Season (2003-2022):")
print(analysis_data.to_string(index=False))
print(f"\nTotal Games in Analysis Period: {analysis_data['game_count'].sum():,}")

## 10. Summary Statistics

In [ ]:
# Overall statistics
stats = pd.read_sql_query(
    """SELECT 
        AVG(pts_home) as avg_home_pts,
        AVG(pts_away) as avg_away_pts,
        AVG(fg_pct_home) as avg_home_fg_pct,
        AVG(fg_pct_away) as avg_away_fg_pct,
        AVG(reb_home) as avg_home_reb,
        AVG(reb_away) as avg_away_reb
       FROM game
       WHERE season_type = 'Regular Season'
         AND season_id >= 22003;""",
    conn
)

print("Overall Statistics (2003-2022):")
print("=" * 50)
print(f"Average Home Points: {stats['avg_home_pts'][0]:.1f}")
print(f"Average Away Points: {stats['avg_away_pts'][0]:.1f}")
print(f"Average Home FG%: {stats['avg_home_fg_pct'][0]:.3f}")
print(f"Average Away FG%: {stats['avg_away_fg_pct'][0]:.3f}")
print(f"Average Home Rebounds: {stats['avg_home_reb'][0]:.1f}")
print(f"Average Away Rebounds: {stats['avg_away_reb'][0]:.1f}")

## 11. Close Database Connection

In [ ]:
conn.close()
print("Database connection closed")

## Summary

This notebook successfully:
1. Downloaded the NBA Basketball Database from Kaggle (697MB)
2. Set up the SQLite database in the project directory
3. Explored the database structure (16 tables)
4. Examined key tables: game, team, other_stats
5. Verified data quality for the analysis period (2003-2022)
6. Documented 60,000+ regular season games across 20 seasons

## Next Steps

The data is now ready for:
- Feature engineering (42 advanced metrics)
- Model training (Elite Ensemble: XGBoost + LightGBM + CatBoost)
- Championship prediction analysis

See `train_elite_model.py` for the machine learning pipeline.